In [19]:
import pandas as pd
import os
import pickle as pkl
from functools import partial
from tqdm import tqdm

In [20]:
tweets_path = "/mlodata1/prakhar/all_available_tweets"
tweets_piped_path = "/mlodata1/prakhar/all_available_tweets_piped"
topics_path = "/mlodata1/prakhar/twitter_covid_insights/insights_All/topics.pkl"
topics = pkl.load(open(topics_path, 'rb'))[0]

# Nb. of tweets per day for normalization
day_flux_path = '/mlodata1/prakhar/twitter_covid_insights/insights_All/day_flux.pkl'
day_flux = pkl.load(open(day_flux_path, 'rb'))

# Read counts
trends_path = '/mlodata1/prakhar/twitter_covid_insights/insights_All/trends.pkl'
trends = pkl.load(open(trends_path, 'rb'))[0]

In [21]:
def has_relevant_hashtag(row, relevant_hashtags):
    if len(set(row['hashtags'].split(',')) & set(relevant_hashtags)) > 0:
        return True
    return False

In [22]:
# workfromhome
relevant_topic = 71
topic_dfs = {relevant_topic: []}
for file in tqdm(os.listdir(tweets_piped_path)):
    piped_tweets = pd.read_parquet(os.path.join(tweets_piped_path, file))
    print(piped_tweets.head(15))
    tweets = pd.read_parquet(os.path.join(tweets_path, file))
    dated_tweets = pd.merge(piped_tweets, tweets[['id', 'created_at']], on='id', how='inner')
    relevant_hashtags = topics[relevant_topic]
    relevant_tweets = dated_tweets[dated_tweets.apply(lambda x: has_relevant_hashtag(x, relevant_hashtags), axis=1)]
    topic_dfs[relevant_topic].append(relevant_tweets.copy())
    break

all_quotes = pd.concat(topic_dfs[relevant_topic])
#     tweets = pd.read_parquet(os.path.join(tweets_path, file))
#     print(tweets.head())
#     cols_to_use = tweets.columns.difference(piped_tweets.columns).tolist()
#     cols_to_use.append('id')
#     merged_tweets = pd.merge(piped_tweets, tweets[cols_to_use], on='id', how='inner')
#     print(merged_tweets.shape)
#     break

  0%|          | 0/685 [00:00<?, ?it/s]

                     id country_code lang  \
0   1178825056782344192           US   en   
1   1178827335069523969           US   en   
2   1178832448312025089           US   en   
3   1178836184660938753         None   en   
4   1178836409643393024           US   en   
5   1178849341315960832         None   en   
6   1178854466662977536           US   en   
7   1178854564998438913         None   en   
8   1178884561608028160         None   en   
9   1178886231217905665           KE   en   
10  1178890230440443905         None   in   
11  1178892222864867328           IN   en   
12  1178894742186278912           GB   en   
13  1178921397147426821           US   en   
14  1178939249586049025           GB   en   

                                            hashtags  \
0                                                      
1                                                      
2                                                      
3                #cgseoul,#bitcoinsv,#bitcoingenesis   

In [118]:
all_quotes['date'] = all_quotes['created_at'].dt.date

In [119]:
all_quotes.head()

,id,country_code,lang,hashtags,cleaned_text,created_at,date
91,1182036308274925568,None,en,"#digitalid,#vaccines",ICYMI user_placeholder and partners launch pro...,2019-10-09 20:53:11+00:00,2019-10-09
54,1188483102416482305,GB,en,"#vaccinehesitancy,#whs2019",Health professionals are key to combatting say...,2019-10-27 15:50:27+00:00,2019-10-27
70,1189645417002217472,US,en,"#meru,#kenya,#cancer,#africa,#vaccines",user_placeholder user_placeholder I'll bet tha...,2019-10-30 20:49:04+00:00,2019-10-30
0,1191869712784859137,US,en,"#vaccines,#redpill",Thread: 💉 This is one of the hardest subjects ...,2019-11-06 00:07:38+00:00,2019-11-06
40,1192039384440287232,US,en,#vaccine,.user_placeholder you & I have have done user_...,2019-11-06 11:21:50+00:00,2019-11-06


In [125]:
all_quotes.groupby(['country_code', 'date'], as_index=False)['id'].count()

,country_code,date,id
0,AE,2020-01-26,1
1,AU,2020-01-22,5
2,AU,2020-01-23,2
3,AU,2020-01-24,3
4,AU,2020-01-25,5
...,...,...,...
175,US,2020-01-26,47
176,US,2020-01-27,69
177,US,2020-01-28,69
178,ZA,2020-01-24,1


In [126]:
all_quotes[all_quotes['country_code'] == 'JP']

,id,country_code,lang,hashtags,cleaned_text,created_at,date
2549,1219460754027376640,JP,en,"#vaccine,#coronavirus,#wuhan,#pneumonia,#ncov",for new Chinese in the works url_placeholder v...,2020-01-21 03:24:34+00:00,2020-01-21
49255,1219965411728465920,JP,en,"#russia,#vaccine,#coronavirus",Developing Against Detected in China - Head of...,2020-01-22 12:49:54+00:00,2020-01-22
54673,1219981964532256774,JP,en,"#russia,#vaccine,#coronavirus,#pneumonia,#ncov",is developing against - RIA cites regulator ur...,2020-01-22 13:55:41+00:00,2020-01-22
66180,1220592779492909056,JP,en,"#coronavirus,#vaccine,#jwo2020",Too fast.🤔 I will not get a coronavirus vacci...,2020-01-24 06:22:50+00:00,2020-01-24
249877,1220818975010906112,JP,en,"#frctlib,#2019ncov,#cbrne,#vaccine,#人口削減,#優生学,...",Bill Gates warns that the next pandemic diseas...,2020-01-24 21:21:40+00:00,2020-01-24
139931,1221023215159394305,JP,en,"#frctlib,#2019ncov,#cbrne,#vaccine,#人口削減,#bill...",CEPI (funded by Gates) Grants INOVIO $9M To De...,2020-01-25 10:53:14+00:00,2020-01-25
149267,1221033308454182912,JP,en,"#wuhanvirus,#vaccine,#staysafe,#beprepared,#co...",user_placeholder The nxt few weeks will see sp...,2020-01-25 11:33:21+00:00,2020-01-25
169121,1221054144821088256,JP,en,"#wuhanvirus,#vaccine,#staysafe,#beprepared,#pr...",user_placeholder Nxt few weeks will spread rap...,2020-01-25 12:56:08+00:00,2020-01-25
175961,1221061096754180099,JP,en,"#wuhanvirus,#vaccine,#staysafe,#beprepared,#ch...",Nxt few weeks will spread rapidly For protecti...,2020-01-25 13:23:46+00:00,2020-01-25
106821,1221340518539378688,JP,en,"#event201,#pirbrightinstitute,#vaccines,#coron...",bill&melinda gates foundation who co-hosted ar...,2020-01-26 07:54:05+00:00,2020-01-26


In [127]:
data = {'dates': pd.date_range('2020-04-01', '2021-09-30'), 'counts': [0 for date in pd.date_range('2020-04-01', '2021-09-30')]}
all_vals = pd.DataFrame.from_dict(data)
all_vals

,dates,counts
0,2020-04-01,0
1,2020-04-02,0
2,2020-04-03,0
3,2020-04-04,0
4,2020-04-05,0
...,...,...
543,2021-09-26,0
544,2021-09-27,0
545,2021-09-28,0
546,2021-09-29,0
